In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans
from scipy.linalg import pinv

In [2]:
# Environment Setup
# ----------------------------
np.random.seed(42)  # For reproducibility
n_samples = 200
n_features = 5

# Generate random dataset
X = np.random.rand(n_samples, n_features)
y = np.random.rand(n_samples)

# ----------------------------
# Preprocessing
# ----------------------------
# Normalize features (Min-Max Scaling)
X_min = X.min(axis=0)
X_max = X.max(axis=0)
X_scaled = (X - X_min) / (X_max - X_min)

# ----------------------------
# Train-Test Split
# ----------------------------
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# ----------------------------
# Radial Basis Function Network (RBFN) Implementation
# ----------------------------
class RBFN:
    def __init__(self, n_centers, spread=1.0):
        self.n_centers = n_centers
        self.spread = spread
        self.centers = None
        self.weights = None

    def _rbf(self, x, center):
        return np.exp(-np.linalg.norm(x - center) ** 2 / (2 * self.spread ** 2))

    def _calculate_interpolation_matrix(self, X):
        G = np.zeros((X.shape[0], self.n_centers))
        for i, sample in enumerate(X):
            for j, center in enumerate(self.centers):
                G[i, j] = self._rbf(sample, center)
        return G

    def fit(self, X, y):
        # Use KMeans to find centers
        kmeans = KMeans(n_clusters=self.n_centers, random_state=42, n_init=10)
        kmeans.fit(X)
        self.centers = kmeans.cluster_centers_

        # Calculate interpolation matrix and compute weights using pseudo-inverse
        G = self._calculate_interpolation_matrix(X)
        self.weights = pinv(G).dot(y)

    def predict(self, X):
        G = self._calculate_interpolation_matrix(X)
        return G.dot(self.weights)

In [3]:
# Train Base Model (RBFN)
# ----------------------------
n_centers = 10
spread = 0.5
rbfn = RBFN(n_centers=n_centers, spread=spread)
rbfn.fit(X_train, y_train)

# ----------------------------
# Planning (Simulated Experience Using RBF Activations)
# ----------------------------
# Visualize activations for the first test sample (simulation)
activations = rbfn._calculate_interpolation_matrix(X_test[:1])
print("Simulated Activations for First Test Sample:", activations)

# ----------------------------
# Fine-Tune Model (Re-train with Adjusted Spread)
# ----------------------------
rbfn.spread = 0.7  # Adjust spread for fine-tuning
rbfn.fit(X_train, y_train)

# ----------------------------
# Evaluate
# ----------------------------
y_pred = rbfn.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Test MSE:", mse)

# ----------------------------
# Deploy Policy (Mock Deployment)
# ----------------------------
def deploy_model(model, X_new):
    """Simulate deployment by making predictions on new data."""
    predictions = model.predict(X_new)
    print("Deployed Model Predictions:", predictions)

# Simulate deployment on random new data
X_new = np.random.rand(5, n_features)
X_new_scaled = (X_new - X_min) / (X_max - X_min)
deploy_model(rbfn, X_new_scaled)

Simulated Activations for First Test Sample: [[0.11381707 0.03489891 0.06274705 0.42453104 0.46368137 0.17832662
  0.11982774 0.18912457 0.45570939 0.09481606]]
Test MSE: 0.07729115657449144
Deployed Model Predictions: [0.49497707 0.52273316 0.68400738 0.62026077 0.51960597]
